In [1]:
try:
    import google.colab
    IN_COLAB = True
except:
    IN_COLAB = False

import os, sys, gc

if IN_COLAB:
    %pip install transformer_lens

    from google.colab import drive
    drive.mount("/content/gdrive", force_remount=True)
    %cd /content/gdrive/MyDrive/mres/model_harvesting

Mounted at /content/gdrive
/content/gdrive/MyDrive/mres/model_harvesting


In [2]:
import torch as t
from torch import nn, Tensor
import torch.nn.functional as F

import numpy as np
import pandas as pd

from transformer_lens import HookedTransformer

from tqdm.notebook import tqdm, trange
from IPython.display import clear_output

In [3]:
device = t.device("cuda" if t.cuda.is_available() else "cpu")

def free_mem(vars):
    for v in vars: del v
    gc.collect()
    t.cuda.empty_cache()

In [4]:
models = {
    "llama2": "Llama-2-7b-chat",
    "llama3": "meta-llama/Meta-Llama-3-8B-Instruct",
    "mistral": "mistral-7b-instruct"
}

model_name = "mistral"
model = HookedTransformer.from_pretrained(models[model_name], device=device); model.eval()

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

...

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

Loaded pretrained model mistral-7b-instruct into HookedTransformer


HookedTransformer(
  (embed): Embed()
  (hook_embed): HookPoint()
  (blocks): ModuleList(
    (0-31): 32 x TransformerBlock(
      (ln1): RMSNormPre(
        (hook_scale): HookPoint()
        (hook_normalized): HookPoint()
      )
      (ln2): RMSNormPre(
        (hook_scale): HookPoint()
        (hook_normalized): HookPoint()
      )
      (attn): GroupedQueryAttention(
        (hook_k): HookPoint()
        (hook_q): HookPoint()
        (hook_v): HookPoint()
        (hook_z): HookPoint()
        (hook_attn_scores): HookPoint()
        (hook_pattern): HookPoint()
        (hook_result): HookPoint()
        (hook_rot_k): HookPoint()
        (hook_rot_q): HookPoint()
      )
      (mlp): GatedMLP(
        (hook_pre): HookPoint()
        (hook_pre_linear): HookPoint()
        (hook_post): HookPoint()
      )
      (hook_attn_in): HookPoint()
      (hook_q_input): HookPoint()
      (hook_k_input): HookPoint()
      (hook_v_input): HookPoint()
      (hook_mlp_in): HookPoint()
      (hook_att

In [5]:
tk_scores = [model.to_tokens(str(i))[:, -1] for i in range(1, 6)]

aspects = ["coherence", "consistency", "fluency", "relevance"]

### newsroom

In [6]:
prompts = pd.read_json("prompts/newsroom-theirs-score.jsonl", orient="records", lines=True)
for aspect in aspects:
    scores = []
    for i in trange(len(prompts), desc=aspect):
        prompt = prompts.at[prompts.index[i], f"prompt-{aspect}"]
        tks = model.to_tokens(prompt)
        with t.no_grad(): logits = model(tks, return_type="logits")
        logits = logits[:, -1, tk_scores]
        score = t.argmax(logits[0]).item() + 1
        scores.append(score)
    prompts["score"] = scores
    prompts.to_json(f"{model_name}/scores/{aspect}.jsonl", orient="records", lines=True)

coherence:   0%|          | 0/420 [00:00<?, ?it/s]

consistency:   0%|          | 0/420 [00:00<?, ?it/s]

fluency:   0%|          | 0/420 [00:00<?, ?it/s]

relevance:   0%|          | 0/420 [00:00<?, ?it/s]